In [0]:
df_product = spark.read.parquet("/mnt/bronze/production/product/20211208")
df_productcat = spark.read.parquet("/mnt/bronze/production/product_category/20211208")
df_productsubcat = spark.read.parquet("/mnt/bronze/production/product_subcategory/20211208")

In [0]:
display(df_product)

ProductID,Name,ProductNumber,MakeFlag,FinishedGoodsFlag,Color,SafetyStockLevel,ReorderPoint,StandardCost,ListPrice,Size,SizeUnitMeasureCode,WeightUnitMeasureCode,Weight,DaysToManufacture,ProductLine,Class,Style,ProductSubcategoryID,ProductModelID,SellStartDate,SellEndDate,DiscontinuedDate,rowguid,ModifiedDate
1,Adjustable Race,AR-5381,false,false,null,1000,750,0.0000,0.0000,null,null,null,null,0,null,null,null,null,null,2008-04-30T00:00:00.000+0000,null,null,694215b7-08f7-4c0d-acb1-d734ba44c0c8,2014-02-08T10:01:36.827+0000
2,Bearing Ball,BA-8327,false,false,null,1000,750,0.0000,0.0000,null,null,null,null,0,null,null,null,null,null,2008-04-30T00:00:00.000+0000,null,null,58ae3c20-4f3a-4749-a7d4-d568806cc537,2014-02-08T10:01:36.827+0000
3,BB Ball Bearing,BE-2349,true,false,null,800,600,0.0000,0.0000,null,null,null,null,1,null,null,null,null,null,2008-04-30T00:00:00.000+0000,null,null,9c21aed2-5bfa-4f18-bcb8-f11638dc2e4e,2014-02-08T10:01:36.827+0000
4,Headset Ball Bearings,BE-2908,false,false,null,800,600,0.0000,0.0000,null,null,null,null,0,null,null,null,null,null,2008-04-30T00:00:00.000+0000,null,null,ecfed6cb-51ff-49b5-b06c-7d8ac834db8b,2014-02-08T10:01:36.827+0000
316,Blade,BL-2036,true,false,null,800,600,0.0000,0.0000,null,null,null,null,1,null,null,null,null,null,2008-04-30T00:00:00.000+0000,null,null,e73e9750-603b-4131-89f5-3dd15ed5ff80,2014-02-08T10:01:36.827+0000
317,LL Crankarm,CA-5965,false,false,Black,500,375,0.0000,0.0000,null,null,null,null,0,null,L,null,null,null,2008-04-30T00:00:00.000+0000,null,null,3c9d10b7-a6b2-4774-9963-c19dcee72fea,2014-02-08T10:01:36.827+0000
318,ML Crankarm,CA-6738,false,false,Black,500,375,0.0000,0.0000,null,null,null,null,0,null,M,null,null,null,2008-04-30T00:00:00.000+0000,null,null,eabb9a92-fa07-4eab-8955-f0517b4a4ca7,2014-02-08T10:01:36.827+0000
319,HL Crankarm,CA-7457,false,false,Black,500,375,0.0000,0.0000,null,null,null,null,0,null,null,null,null,null,2008-04-30T00:00:00.000+0000,null,null,7d3fd384-4f29-484b-86fa-4206e276fe58,2014-02-08T10:01:36.827+0000
320,Chainring Bolts,CB-2903,false,false,Silver,1000,750,0.0000,0.0000,null,null,null,null,0,null,null,null,null,null,2008-04-30T00:00:00.000+0000,null,null,7be38e48-b7d6-4486-888e-f53c26735101,2014-02-08T10:01:36.827+0000
321,Chainring Nut,CN-6137,false,false,Silver,1000,750,0.0000,0.0000,null,null,null,null,0,null,null,null,null,null,2008-04-30T00:00:00.000+0000,null,null,3314b1d7-ef69-4431-b6dd-dc75268bd5df,2014-02-08T10:01:36.827+0000


In [0]:
display(df_productcat)

ProductCategoryID,Name,rowguid,ModifiedDate
1,Bikes,cfbda25c-df71-47a7-b81b-64ee161aa37c,2008-04-30T00:00:00.000+0000
2,Components,c657828d-d808-4aba-91a3-af2ce02300e9,2008-04-30T00:00:00.000+0000
3,Clothing,10a7c342-ca82-48d4-8a38-46a2eb089b74,2008-04-30T00:00:00.000+0000
4,Accessories,2be3be36-d9a2-4eee-b593-ed895d97c2a6,2008-04-30T00:00:00.000+0000


In [0]:
display(df_productsubcat)

ProductSubcategoryID,ProductCategoryID,Name,rowguid,ModifiedDate
1,1,Mountain Bikes,2d364ade-264a-433c-b092-4fcbf3804e01,2008-04-30T00:00:00.000+0000
2,1,Road Bikes,000310c0-bcc8-42c4-b0c3-45ae611af06b,2008-04-30T00:00:00.000+0000
3,1,Touring Bikes,02c5061d-ecdc-4274-b5f1-e91d76bc3f37,2008-04-30T00:00:00.000+0000
4,2,Handlebars,3ef2c725-7135-4c85-9ae6-ae9a3bdd9283,2008-04-30T00:00:00.000+0000
5,2,Bottom Brackets,a9e54089-8a1e-4cf5-8646-e3801f685934,2008-04-30T00:00:00.000+0000
6,2,Brakes,d43ba4a3-ef0d-426b-90eb-4be4547dd30c,2008-04-30T00:00:00.000+0000
7,2,Chains,e93a7231-f16c-4b0f-8c41-c73fdec62da0,2008-04-30T00:00:00.000+0000
8,2,Cranksets,4f644521-422b-4f19-974a-e3df6102567e,2008-04-30T00:00:00.000+0000
9,2,Derailleurs,1830d70c-aa2a-40c0-a271-5ba86f38f8bf,2008-04-30T00:00:00.000+0000
10,2,Forks,b5f9ba42-b69b-4fdd-b2ec-57fb7b42e3cf,2008-04-30T00:00:00.000+0000


In [0]:
df_product.registerTempTable("product")
df_productcat.registerTempTable("productcat")
df_productsubcat.registerTempTable("productsubcat")

In [0]:
df_join = spark.sql("""
select 
P.ProductID ProdutoID,
P.Name NomeProduto,
PSC.ProductCategoryID ProdutoCategoriaID,
PSC.Name NomeSubProduto,
PC.Name NomeCategoria
from product P
left join productsubcat PSC
  on P.ProductSubcategoryID = PSC.ProductSubcategoryID
left join productcat PC
  on PC.ProductCategoryID = PSC.ProductCategoryID""")

In [0]:
display(df_join)

ProdutoID,NomeProduto,ProdutoCategoriaID,NomeSubProduto,NomeCategoria
1,Adjustable Race,null,null,null
2,Bearing Ball,null,null,null
3,BB Ball Bearing,null,null,null
4,Headset Ball Bearings,null,null,null
316,Blade,null,null,null
317,LL Crankarm,null,null,null
318,ML Crankarm,null,null,null
319,HL Crankarm,null,null,null
320,Chainring Bolts,null,null,null
321,Chainring Nut,null,null,null


In [0]:
from pyspark.sql.functions import when
df2 = df_join.withColumn("NomeCategoria", when(df_join.NomeCategoria.isNull(), "Outros")
                                 .otherwise(df_join.NomeCategoria))
df2 = df2.withColumn("NomeSubProduto", when(df2.NomeSubProduto.isNull(), "Outros")
                                 .otherwise(df_join.NomeSubProduto))
df2 = df2.withColumn("ProdutoCategoriaID", when(df2.ProdutoCategoriaID.isNull(), -1)
                                 .otherwise(df2.ProdutoCategoriaID))

In [0]:
display(df2)

ProdutoID,NomeProduto,ProdutoCategoriaID,NomeSubProduto,NomeCategoria
1,Adjustable Race,-1,Outros,Outros
2,Bearing Ball,-1,Outros,Outros
3,BB Ball Bearing,-1,Outros,Outros
4,Headset Ball Bearings,-1,Outros,Outros
316,Blade,-1,Outros,Outros
317,LL Crankarm,-1,Outros,Outros
318,ML Crankarm,-1,Outros,Outros
319,HL Crankarm,-1,Outros,Outros
320,Chainring Bolts,-1,Outros,Outros
321,Chainring Nut,-1,Outros,Outros


In [0]:
df2.registerTempTable("final")


In [0]:
%sql
select
ProdutoID,
upper(NomeProduto) as Produto,
ProdutoCategoriaID,
NomeSubProduto,
NomeCategoria
from
final

ProdutoID,upper(NomeProduto),ProdutoCategoriaID,NomeSubProduto,NomeCategoria
1,ADJUSTABLE RACE,-1,Outros,Outros
2,BEARING BALL,-1,Outros,Outros
3,BB BALL BEARING,-1,Outros,Outros
4,HEADSET BALL BEARINGS,-1,Outros,Outros
316,BLADE,-1,Outros,Outros
317,LL CRANKARM,-1,Outros,Outros
318,ML CRANKARM,-1,Outros,Outros
319,HL CRANKARM,-1,Outros,Outros
320,CHAINRING BOLTS,-1,Outros,Outros
321,CHAINRING NUT,-1,Outros,Outros


In [0]:
sql = """
select
ProdutoID,
upper(NomeProduto) as NomeProduto,
ProdutoCategoriaID,
NomeSubProduto,
NomeCategoria
from
final"""
df_upper = spark.sql(sql)
display(df_upper)

ProdutoID,NomeProduto,ProdutoCategoriaID,NomeSubProduto,NomeCategoria
1,ADJUSTABLE RACE,-1,Outros,Outros
2,BEARING BALL,-1,Outros,Outros
3,BB BALL BEARING,-1,Outros,Outros
4,HEADSET BALL BEARINGS,-1,Outros,Outros
316,BLADE,-1,Outros,Outros
317,LL CRANKARM,-1,Outros,Outros
318,ML CRANKARM,-1,Outros,Outros
319,HL CRANKARM,-1,Outros,Outros
320,CHAINRING BOLTS,-1,Outros,Outros
321,CHAINRING NUT,-1,Outros,Outros


In [0]:
df_upper.write.format("Delta").save("/mnt/silver/production/TBProdutoCategoria")